In [ ]:
import csv
import json
import requests
import numpy as np
import math
import time
import os

In [ ]:
Path = os.getcwd() + "\credentials.txt"
def getCredentials(path=Path):
    with open(path,"r") as file:
        contents = file.read()
        creds = contents.split("\n")
        user = creds[0]
        token = creds[1]
    return user,token

In [ ]:
org = ""
repoList = []
teamsFile=""
parameters={"state":"all"}
url_headers = {
                "Accept" : "application/vnd.github.cloak-preview",
            }
user,token = getCredentials()
sleep_time = 60

In [ ]:
def getTeamMembers(Headers=url_headers,Auth=(user,token)):
    teamUrl = "https://api.github.com/orgs/" +org+"/teams"
    teamMembers={}
    response = requests.get(teamUrl,headers=Headers,auth=Auth)
    result = response.json()
    for team in result:
        url = "https://api.github.com/orgs/"+ org+ "/teams/"+team["slug"]+"/members?per_page=100"
        members = requests.get(url,headers=Headers,auth=Auth)
        members = members.json()
        for member in members:
            response = requests.get(member["url"],headers=Headers,auth=Auth)
            result = response.json()
            teamMembers[member["login"]] = team["name"]
    return teamMembers
# teamMembers = getTeamMembers()


In [ ]:
def getTeams():
    memberToTeam = {}
    path = os.getcwd() + "\\" + teamsFile+".json"
    with open(path,"r") as file:
        contents = file.read()
        data = json.loads(contents)
    for item in data:
        memberToTeam[item["username"]] = item["teams"] 
    return memberToTeam

In [ ]:
def getNextURL(url,Params={},Headers=url_headers,Auth=(user,token)):
    response = requests.get(url,headers=Headers,auth=Auth,params=Params)
    link = response.headers["Link"]
    links = link.split(",")
    linkHash = {}
    for item in links:
        array = item.split(";")
        linkHash[array[1].replace('"',"").replace(" ","")] = array[0].replace("<","").replace(">","")
    return(linkHash["rel=next"])

In [ ]:

# commits_url =   "https://api.github.com/repos/" + org + "/" +repo+ "/commits?sha=master&per_page=100"
def getPages(url,limit=1000,params={},Headers=url_headers):
    urls = []
    urls.append(url)
    for x in range(1,limit):
        try:
            url = getNextURL(urls[x-1],params,Headers)
            if url == "" or url== None:
                break
            else:
                urls.append(url)
        except:
            break         
    print(f"total pages: {len(urls)}")
    return urls


In [ ]:
def getCommitLinks(url,Headers=url_headers,Auth=(user,token)):
    urls = []
    pages = getPages(url)
    for page in pages:
        response = requests.get(page,headers=Headers,auth=Auth)
        result = response.json()
#             print(json.dumps(result,indent=4))
        for commitItem in result:
            commitURL = commitItem["url"]
            urls.append(commitURL)
    print(f"total commit items: {len(urls)}")
    return urls

# commitURLs = getCommitLinks()

In [ ]:
def get_commit_data(commitURLs,memberToTeam):
    data = []
    for url in commitURLs:
        item = {}
        item["rnd"] = round(np.random.rand(1)[0],3)
        response = requests.get(url,headers=url_headers,auth=(user,token))
        result = response.json()
        item["timestamp"] = result["commit"]["committer"]["date"]
        personID = "n/a"
        key1 = "author"
        if key1 in result.keys() and result[key1] != None:
            key2 = "login"
            if key2 in result[key1].keys() and result[key1][key2] != None:
                personID = result["author"]["login"]
        else:
            key1 = "login"
            if key1 in result["commit"]["committer"].keys() and result["commit"]["committer"][key1]!=None:
                personID = result["commit"]["committer"]["login"]
            else:
                key1 = "author"
                if key1 in result["commit"].keys() and result["commit"]["author"]["name"]!= None:
                    personID = result["commit"]["author"]["name"]
                else:
                    personID = "n/a"
        item["personID"] = personID
        
        team = "n/a"
        if personID in memberToTeam:
            team = memberToTeam[personID]
            if team == None or team=="" or team == "null":
                team = "n/a"
            
        item["teamID"] =  team
        item["type"] = "commit"
        item["commitMessage"] = result["commit"]["message"]
        item["commitAdditions"] = result["stats"]["additions"]
        item["commitDeletions"] = result["stats"]["deletions"]
    
        item["isDoc"] = False
        for file in result["files"]:
            filename = file["filename"]
            name,ext = os.path.splitext(filename)
            types = [".md",".SAMPLE",".json",".sh"]
            names = [".gitignore"]
            if ext in types or name in names:
                item["isDoc"] = True
            else:
                item["isDoc"] = False
                break
                
        item["commitFilesTouched"] = []
        for file in result["files"]:
            item["commitFilesTouched"].append(file["filename"])
                
        item["url"] = result["html_url"]
        print(item)
        data.append(item)
        print("-------------------------------------------------------------------------------------")
        print("\n")
    return {"commitData" : data }
# commitData = get_commit_data()
# print(commitData)

In [ ]:
def getCSV(commits,repo):
        with open(repo+".csv", mode="w+",encoding="utf-8",newline='') as file:
            writer = csv.writer(file, delimiter=",")
            writer.writerow(["rnd","doc only","timestamp","personID","team","type","commit message",
                            "additions","deletions","files changed","url"])
    
        with open(repo+".csv",mode="a",encoding="utf-8",newline='') as file:
            writer = csv.writer(file,delimiter=",")
            for key in commits:
                commitsArray = commits[key]
                for item in commitsArray:
                    writer.writerow([item["rnd"],item["isDoc"],item["timestamp"],item["personID"],item["teamID"],
                                    item["type"],
                                    "\""+item["commitMessage"]+"\"",
                                    item["commitAdditions"],item["commitDeletions"],
                                    item["commitFilesTouched"],item["url"]])
# getCommitsOnly(commitData)

In [ ]:
def getFiles():
    for repo in repoList:
        while True:
            try:
                print(repo)
                url =   "https://api.github.com/repos/" + org + "/" + repo + "/commits?sha=master&per_page=100"
                teams = getTeams()
                commitURLs = getCommitLinks(url)
                commitData = get_commit_data(commitURLs,teams)
                getCSV(commitData,repo)
                break
            except:
                print("\n\n\nConnection timed out..............................................")
                print(f"sleeping for {sleep_time}s....")
                time.sleep(sleep_time)
                continue
getFiles()